In [2]:
from tqdm import tqdm
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.features import shapes
from shapely.geometry import shape, Polygon

Resample tif to be 3m perhaps?

In [3]:
# === Paths ===
input_ndvi_tif = "D:/terrain_generation_project/NAIP_processed/naip_ndvi_1_8.tif"
output_gpkg = "D:/terrain_generation_project/NAIP_processed/vegetation_polygons.gpkg"
layer_name = "vegetation"

In [ ]:
# === Parameters ===
ndvi_min = 0.3
ndvi_max = 0.8
min_area_m2 = 1.0
simplify_tolerance = 0.5  # meters

# === Load NDVI ===
with rasterio.open(input_ndvi_tif) as src:
    ndvi = src.read(1)  # Single band
    transform = src.transform
    crs = src.crs

# === Create mask for NDVI between thresholds ===
mask = (ndvi >= ndvi_min) & (ndvi <= ndvi_max)

# === Estimate progress length ===
estimated_total = np.count_nonzero(mask)

# === Stream shapes with cleanup ===
polygons = []
with tqdm(total=estimated_total, desc="Extracting vegetation polygons") as pbar:
    for geom, val in shapes(ndvi.astype(np.float32), mask=mask, transform=transform):
        if ndvi_min <= val <= ndvi_max:
            poly = shape(geom)

            # Skip very small polygons
            if poly.area < min_area_m2:
                pbar.update(1)
                continue

            # Remove tiny holes
            if isinstance(poly, Polygon):
                new_interiors = [
                    ring for ring in poly.interiors
                    if Polygon(ring).area >= min_area_m2
                ]
                poly = Polygon(poly.exterior, new_interiors)

            # Simplify geometry
            simplified_poly = poly.simplify(simplify_tolerance, preserve_topology=True)

            polygons.append(simplified_poly)
        pbar.update(1)

# === Create GeoDataFrame and save ===
gdf = gpd.GeoDataFrame(geometry=polygons, crs=crs)
gdf.to_file(output_gpkg, layer=layer_name, driver="GPKG")

print(f"Saved {len(gdf)} simplified vegetation polygons to '{output_gpkg}' in layer '{layer_name}'.")

Extracting vegetation polygons:   0%|          | 0/143631423 [00:00<?, ?it/s]